In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import gc
import time
from collections import defaultdict, Counter

# PyTorch & ML Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, OneCycleLR
import torchvision.transforms as transforms
from torchvision.models import efficientnet_v2_s, efficientnet_v2_m
import timm

# Scientific Computing
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# Image Processing
from PIL import Image
import cv2

# Utilities
import random
import json
from pathlib import Path

# 경고 메시지 숨기기
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 시드 고정 (재현성 보장)
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

print("🔥 건설용 자갈 암석 분류 AI - 모델 학습 시작!")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print("=" * 60)

c:\Users\wang\anaconda3\envs\yolo\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔥 건설용 자갈 암석 분류 AI - 모델 학습 시작!
PyTorch 버전: 2.2.2+cu121
CUDA 사용 가능: True
GPU: NVIDIA GeForce RTX 4070


In [2]:

# ============================================================================
# 🏆 상위 팀들의 핵심 전략 통합 설정
# 1위 팀: 4개 모델 앙상블 (EfficientNetV2-S/M + RegNetY + TinyViT)
# 2위 팀: InternImage + Hard Negative Sampling
# 3위 팀: ConvNeXt 
# 4위 팀: 안정적인 전이학습 전략
# ============================================================================

# 경로 설정
BASE_PATH = r"D:\data\stones\open"
TRAIN_PATH = os.path.join(BASE_PATH, "train")
TEST_PATH = os.path.join(BASE_PATH, "test")
SUBMISSION_PATH = os.path.join(BASE_PATH, "sample_submission.csv")

# 실험 결과 저장 경로
EXPERIMENT_DIR = Path("../experiments")
EXPERIMENT_DIR.mkdir(exist_ok=True)

# 모델 저장 경로
MODEL_DIR = Path("../models")
MODEL_DIR.mkdir(exist_ok=True)

print("📂 경로 설정 완료:")
print(f"훈련 데이터: {TRAIN_PATH}")
print(f"테스트 데이터: {TEST_PATH}")
print(f"실험 결과: {EXPERIMENT_DIR}")
print(f"모델 저장: {MODEL_DIR}")

# 클래스 정보 (이전 분석 결과 활용)
CLASS_NAMES = ['Andesite', 'Basalt', 'Etc', 'Gneiss', 'Granite', 'Mud_Sandstone', 'Weathered_Rock']
CLASS_COUNTS = {
    'Andesite': 43802,
    'Basalt': 26810,
    'Etc': 15935,
    'Gneiss': 73914,
    'Granite': 92923,
    'Mud_Sandstone': 89467,
    'Weathered_Rock': 37169
}

NUM_CLASSES = len(CLASS_NAMES)
class_to_idx = {cls: i for i, cls in enumerate(sorted(CLASS_NAMES))}
idx_to_class = {i: cls for cls, i in class_to_idx.items()}

print(f"\n🏷️ 클래스 정보:")
print(f"클래스 수: {NUM_CLASSES}")
print(f"클래스 목록: {sorted(CLASS_NAMES)}")

# 하이퍼파라미터 설정 (상위 팀들의 최적 설정 통합)
HYPERPARAMETERS = {
    # 기본 설정
    'image_size': 224,          # 1-4위 팀 공통 사용
    'batch_size': 8,           # GPU 메모리 효율성 고려
    'num_epochs': 50,           # 충분한 학습 시간
    'num_workers': 0,           # Windows 호환성
    'pin_memory': True,
    
    # 학습률 설정 (1위 팀 전략)
    'lr': 3e-4,                 # AdamW 최적 학습률
    'weight_decay': 1e-4,       # 정규화 강도
    'warmup_epochs': 5,         # 학습률 워밍업
    
    # 증강 설정 (2위 팀 전략)
    'cutmix_alpha': 1.0,        # CutMix 강도
    'mixup_alpha': 0.2,         # Mixup 강도
    'cutmix_prob': 0.5,         # CutMix 적용 확률
    'mixup_prob': 0.5,          # Mixup 적용 확률
    
    # 샘플링 설정 (2위 팀 Hard Negative)
    'hard_negative_ratio': 0.2,  # 배치 중 Hard Sample 비율
    'hard_memory_size': 1000,     # Hard Sample 메모리 크기
    'loss_threshold': 1.5,        # Hard Sample 기준 손실값
    
    # K-Fold 설정 (2-4위 팀 공통)
    'kfold_splits': 5,           # 5-fold 교차검증
    'kfold_seed': 42,
    
    # 앙상블 설정 (1위 팀 전략)
    'ensemble_models': [
        'tf_efficientnetv2_s.in21k_ft_in1k',  # 1위 팀 사용
        'tf_efficientnetv2_m.in21k_ft_in1k',  # 1위 팀 사용
        'convnext_tiny.fb_in22k_ft_in1k',     # 3위 팀 사용
        'vit_tiny_patch16_224.augreg_in21k_ft_in1k'  # Vision Transformer
    ]
}

print(f"\n⚙️ 하이퍼파라미터 설정:")
for key, value in HYPERPARAMETERS.items():
    if isinstance(value, list):
        print(f"{key}: {len(value)}개 모델")
    else:
        print(f"{key}: {value}")

# GPU/CPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n💻 연산 장치: {device}")

if device.type == 'cuda':
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    
print("=" * 60)

📂 경로 설정 완료:
훈련 데이터: D:\data\stones\open\train
테스트 데이터: D:\data\stones\open\test
실험 결과: ..\experiments
모델 저장: ..\models

🏷️ 클래스 정보:
클래스 수: 7
클래스 목록: ['Andesite', 'Basalt', 'Etc', 'Gneiss', 'Granite', 'Mud_Sandstone', 'Weathered_Rock']

⚙️ 하이퍼파라미터 설정:
image_size: 224
batch_size: 8
num_epochs: 50
num_workers: 0
pin_memory: True
lr: 0.0003
weight_decay: 0.0001
warmup_epochs: 5
cutmix_alpha: 1.0
mixup_alpha: 0.2
cutmix_prob: 0.5
mixup_prob: 0.5
hard_negative_ratio: 0.2
hard_memory_size: 1000
loss_threshold: 1.5
kfold_splits: 5
kfold_seed: 42
ensemble_models: 4개 모델

💻 연산 장치: cuda
GPU 메모리: 12.0 GB


In [3]:
# ============================================================================
# 🎨 데이터 변환 및 증강 (상위 팀들의 베스트 프랙티스 통합)
# ============================================================================

class AdvancedTransforms:
    """상위 팀들의 데이터 변환 전략을 통합한 클래스"""
    
    def __init__(self, image_size=224, is_training=True):
        self.image_size = image_size
        self.is_training = is_training
        
    def get_train_transforms(self):
        """훈련용 변환 (2위 팀 + 3위 팀 전략 통합)"""
        return transforms.Compose([
            transforms.Resize((256, 256), interpolation=Image.BICUBIC),  # 품질 보존
            transforms.RandomCrop(self.image_size),                      # 랜덤 크롭
            transforms.RandomHorizontalFlip(p=0.5),                      # 좌우 뒤집기
            transforms.RandomRotation(degrees=15),                       # 회전 (암석 특성 고려)
            transforms.ColorJitter(                                      # 색상 변화 (현장 환경 반영)
                brightness=0.2,
                contrast=0.2, 
                saturation=0.1,
                hue=0.05
            ),
            transforms.RandomApply([                                     # 추가 증강 (확률적 적용)
                transforms.GaussianBlur(kernel_size=3)
            ], p=0.1),
            transforms.ToTensor(),
            transforms.Normalize(                                        # ImageNet 정규화
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225]
            )
        ])
    
    def get_valid_transforms(self):
        """검증용 변환 (증강 없음)"""
        return transforms.Compose([
            transforms.Resize((self.image_size, self.image_size), interpolation=Image.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225]
            )
        ])
    
    def get_test_transforms(self):
        """테스트용 변환 (TTA 포함 가능)"""
        return self.get_valid_transforms()

# CutMix 구현 (2위 팀 전략)
def cutmix_data(x, y, alpha=1.0):
    """CutMix 데이터 증강"""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)
    
    bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)
    x[:, :, bbx1:bbx2, bby1:bby2] = x[index, :, bbx1:bbx2, bby1:bby2]
    
    # 실제 혼합 비율 계산
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size()[-1] * x.size()[-2]))
    
    return x, y, y[index], lam

def rand_bbox(size, lam):
    """CutMix용 랜덤 박스 생성"""
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Mixup 구현 (2위 팀 전략)
def mixup_data(x, y, alpha=0.2):
    """Mixup 데이터 증강"""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)
    
    mixed_x = lam * x + (1 - lam) * x[index]
    
    return mixed_x, y, y[index], lam

# 손실 함수 (혼합 데이터용)
def mixup_criterion(criterion, pred, y_a, y_b, lam):
    """Mixup/CutMix용 손실 함수"""
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# 변환 객체 생성
transform_manager = AdvancedTransforms(image_size=HYPERPARAMETERS['image_size'])

train_transforms = transform_manager.get_train_transforms()
valid_transforms = transform_manager.get_valid_transforms()
test_transforms = transform_manager.get_test_transforms()

print("🎨 데이터 변환 설정 완료:")
print("✅ 훈련용 변환: 고급 증강 포함")
print("✅ 검증용 변환: 증강 없음")
print("✅ CutMix/Mixup: 구현 완료")
print("✅ 암석 이미지 특성 최적화")

# 변환 예시 시각화를 위한 함수
def visualize_transforms(dataset, num_samples=4):
    """데이터 변환 결과 시각화"""
    fig, axes = plt.subplots(2, num_samples, figsize=(15, 6))
    fig.suptitle('데이터 변환 예시', fontsize=16, fontweight='bold')
    
    for i in range(num_samples):
        # 원본 이미지 (검증용 변환만 적용)
        img, label = dataset[i]
        
        # 첫 번째 행: 검증용 변환
        axes[0, i].imshow(img.permute(1, 2, 0) * 0.229 + 0.485)  # 정규화 해제 (근사)
        axes[0, i].set_title(f'Valid Transform\n{idx_to_class[label]}')
        axes[0, i].axis('off')
        
        # 두 번째 행: 훈련용 변환 (별도 적용)
        # 실제로는 랜덤이므로 매번 다름
        axes[1, i].imshow(img.permute(1, 2, 0) * 0.229 + 0.485)  # 정규화 해제 (근사) 
        axes[1, i].set_title(f'Train Transform\n{idx_to_class[label]}')
        axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()

print("🖼️ 시각화 함수 준비 완료")
print("=" * 60)

🎨 데이터 변환 설정 완료:
✅ 훈련용 변환: 고급 증강 포함
✅ 검증용 변환: 증강 없음
✅ CutMix/Mixup: 구현 완료
✅ 암석 이미지 특성 최적화
🖼️ 시각화 함수 준비 완료


In [4]:
# ============================================================================
# 📊 고급 데이터셋 클래스 (Hard Negative Sampling + WeightedRandomSampler 통합)
# ============================================================================

class RockDataset(Dataset):
    """암석 분류를 위한 고급 데이터셋 클래스"""
    
    def __init__(self, data_dir, transform=None, class_to_idx=None, is_training=True):
        self.data_dir = data_dir
        self.transform = transform
        self.is_training = is_training
        self.class_to_idx = class_to_idx or {}
        
        # 이미지 경로와 라벨 수집
        self.samples = []
        self.class_counts = defaultdict(int)
        
        self._load_samples()
        self._compute_weights()
        
    def _load_samples(self):
        """이미지 샘플 로딩"""
        for class_name in os.listdir(self.data_dir):
            class_path = os.path.join(self.data_dir, class_name)
            if not os.path.isdir(class_path):
                continue
                
            class_idx = self.class_to_idx.get(class_name, len(self.class_to_idx))
            if class_name not in self.class_to_idx:
                self.class_to_idx[class_name] = class_idx
            
            for img_name in os.listdir(class_path):
                if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    img_path = os.path.join(class_path, img_name)
                    self.samples.append((img_path, class_idx))
                    self.class_counts[class_idx] += 1
        
        print(f"📊 데이터셋 로딩 완료: {len(self.samples):,}개 샘플")
        for class_name, class_idx in sorted(self.class_to_idx.items()):
            print(f"   {class_name}: {self.class_counts[class_idx]:,}개")
    
    def _compute_weights(self):
        """샘플별 가중치 계산 (WeightedRandomSampler용)"""
        total_samples = len(self.samples)
        num_classes = len(self.class_to_idx)
        
        # 제곱근 역빈도 가중치 (이전 분석에서 최적으로 확인됨)
        class_weights = {}
        for class_idx, count in self.class_counts.items():
            weight = np.sqrt(total_samples / (num_classes * count))
            class_weights[class_idx] = weight
        
        # 각 샘플에 가중치 할당
        self.sample_weights = []
        for _, class_idx in self.samples:
            self.sample_weights.append(class_weights[class_idx])
        
        self.sample_weights = torch.DoubleTensor(self.sample_weights)
        print(f"⚖️ 가중치 계산 완료: {self.sample_weights.min():.3f} ~ {self.sample_weights.max():.3f}")
    
    def get_weighted_sampler(self):
        """WeightedRandomSampler 반환"""
        return WeightedRandomSampler(
            weights=self.sample_weights,
            num_samples=len(self.sample_weights),
            replacement=True
        )
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        try:
            # 이미지 로딩
            image = Image.open(img_path).convert('RGB')
            
            # 변환 적용
            if self.transform:
                image = self.transform(image)
            
            return image, label, idx  # idx 추가 (Hard Negative 추적용)
        
        except Exception as e:
            print(f"⚠️ 이미지 로딩 오류: {img_path} - {e}")
            # 다른 샘플로 대체
            return self.__getitem__((idx + 1) % len(self.samples))

# Hard Negative Sampler 클래스 (2위 팀 핵심 기법)
class HardNegativeSampler:
    """Hard Negative Sample을 관리하는 클래스"""
    
    def __init__(self, memory_size=1000, loss_threshold=1.5):
        self.memory_size = memory_size
        self.loss_threshold = loss_threshold
        self.hard_samples = []  # (image, label, loss) 튜플들
        self.sample_indices = []  # 원본 데이터셋 인덱스들
        
    def update(self, images, labels, indices, losses):
        """Hard sample 업데이트"""
        # 높은 손실을 가진 샘플들 찾기
        hard_mask = losses > self.loss_threshold
        
        if hard_mask.sum() > 0:
            hard_images = images[hard_mask]
            hard_labels = labels[hard_mask]
            hard_indices = indices[hard_mask]
            hard_losses = losses[hard_mask]
            
            # 새로운 hard sample들 추가
            for img, label, idx, loss in zip(hard_images, hard_labels, hard_indices, hard_losses):
                self.hard_samples.append((img.cpu(), label.cpu(), loss.item()))
                self.sample_indices.append(idx.item())
            
            # 메모리 크기 제한
            if len(self.hard_samples) > self.memory_size:
                # 손실이 높은 순으로 정렬하고 상위 N개만 유지
                sorted_samples = sorted(
                    zip(self.hard_samples, self.sample_indices),
                    key=lambda x: x[0][2],  # loss 기준 정렬
                    reverse=True
                )
                
                self.hard_samples = [s[0] for s in sorted_samples[:self.memory_size]]
                self.sample_indices = [s[1] for s in sorted_samples[:self.memory_size]]
    
    def sample(self, batch_size, device):
        """Hard sample들에서 배치 샘플링"""
        if len(self.hard_samples) == 0:
            return None, None
        
        # 배치 크기만큼 랜덤 샘플링
        sample_size = min(batch_size, len(self.hard_samples))
        sampled_indices = np.random.choice(len(self.hard_samples), sample_size, replace=False)
        
        images = []
        labels = []
        
        for idx in sampled_indices:
            img, label, _ = self.hard_samples[idx]
            images.append(img)
            labels.append(label)
        
        images = torch.stack(images).to(device)
        labels = torch.stack(labels).to(device)
        
        return images, labels
    
    def __len__(self):
        return len(self.hard_samples)

# 데이터셋 생성
print("📊 데이터셋 생성 중...")

# 훈련 데이터셋
train_dataset = RockDataset(
    data_dir=TRAIN_PATH,
    transform=train_transforms,
    class_to_idx=class_to_idx,
    is_training=True
)

# 검증 데이터셋은 나중에 K-Fold에서 분할
print(f"✅ 훈련 데이터셋 생성 완료: {len(train_dataset):,}개 샘플")

# WeightedRandomSampler 생성
weighted_sampler = train_dataset.get_weighted_sampler()
print("✅ WeightedRandomSampler 생성 완료")

# Hard Negative Sampler 생성
hard_negative_sampler = HardNegativeSampler(
    memory_size=HYPERPARAMETERS['hard_memory_size'],
    loss_threshold=HYPERPARAMETERS['loss_threshold']
)
print("✅ Hard Negative Sampler 생성 완료")

# 기본 DataLoader 생성 (K-Fold에서 재구성됨)
train_loader = DataLoader(
    train_dataset,
    batch_size=HYPERPARAMETERS['batch_size'],
    sampler=weighted_sampler,
    num_workers=HYPERPARAMETERS['num_workers'],
    pin_memory=HYPERPARAMETERS['pin_memory'],
    drop_last=True  # 배치 크기 일관성 유지
)

print(f"✅ DataLoader 생성 완료: {len(train_loader)} 배치")
print("=" * 60)

📊 데이터셋 생성 중...
📊 데이터셋 로딩 완료: 380,020개 샘플
   Andesite: 43,802개
   Basalt: 26,810개
   Etc: 15,935개
   Gneiss: 73,914개
   Granite: 92,923개
   Mud_Sandstone: 89,467개
   Weathered_Rock: 37,169개
⚖️ 가중치 계산 완료: 0.764 ~ 1.846
✅ 훈련 데이터셋 생성 완료: 380,020개 샘플
✅ WeightedRandomSampler 생성 완료
✅ Hard Negative Sampler 생성 완료
✅ DataLoader 생성 완료: 47502 배치


In [5]:
# ============================================================================
# 🧠 모델 아키텍처 (1-4위 팀의 최고 모델들 통합)
# ============================================================================

class ModelFactory:
    """상위 팀들이 사용한 모델들을 생성하는 팩토리 클래스"""
    
    @staticmethod
    def create_model(model_name, num_classes=7, pretrained=True):
        """모델 생성"""
        try:
            if 'efficientnetv2' in model_name:
                # 1위 팀 사용 모델
                model = timm.create_model(
                    model_name, 
                    pretrained=pretrained,
                    num_classes=num_classes
                )
                
            elif 'convnext' in model_name:
                # 3위 팀 사용 모델
                model = timm.create_model(
                    model_name,
                    pretrained=pretrained, 
                    num_classes=num_classes
                )
                
            elif 'vit' in model_name:
                # Vision Transformer (다양한 팀에서 실험)
                model = timm.create_model(
                    model_name,
                    pretrained=pretrained,
                    num_classes=num_classes
                )
                
            elif 'regnety' in model_name:
                # 1위 팀 사용 모델
                model = timm.create_model(
                    model_name,
                    pretrained=pretrained,
                    num_classes=num_classes
                )
                
            else:
                raise ValueError(f"지원하지 않는 모델: {model_name}")
            
            print(f"✅ {model_name} 모델 생성 완료")
            return model
            
        except Exception as e:
            print(f"❌ {model_name} 모델 생성 실패: {e}")
            # 대체 모델로 EfficientNetV2-S 사용
            print("🔄 대체 모델 사용: EfficientNetV2-S")
            return timm.create_model(
                'tf_efficientnetv2_s.in21k_ft_in1k',
                pretrained=True,
                num_classes=num_classes
            )
    
    @staticmethod
    def get_model_info(model):
        """모델 정보 출력"""
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        
        return {
            'total_params': total_params,
            'trainable_params': trainable_params,
            'size_mb': total_params * 4 / 1024 / 1024  # 대략적인 크기 (float32 기준)
        }

# 모델별 설정
MODEL_CONFIGS = {
    'tf_efficientnetv2_s.in21k_ft_in1k': {
        'name': 'EfficientNetV2-S',
        'team': '1위 팀 사용',
        'description': '효율성과 성능의 균형',
        'lr_multiplier': 1.0
    },
    'tf_efficientnetv2_m.in21k_ft_in1k': {
        'name': 'EfficientNetV2-M',
        'team': '1위 팀 사용', 
        'description': '더 큰 모델, 높은 성능',
        'lr_multiplier': 0.8
    },
    'convnext_tiny.fb_in22k_ft_in1k': {
        'name': 'ConvNeXt-Tiny',
        'team': '3위 팀 사용',
        'description': 'CNN + Transformer 하이브리드',
        'lr_multiplier': 1.2
    },
    'vit_tiny_patch16_224.augreg_in21k_ft_in1k': {
        'name': 'ViT-Tiny',
        'team': '다양한 팀 실험',
        'description': 'Vision Transformer',
        'lr_multiplier': 1.5
    }
}

# 사용 가능한 모델 확인
print("🧠 사용 가능한 모델 확인 중...")
available_models = []

for model_name in HYPERPARAMETERS['ensemble_models']:
    try:
        # 모델 생성 테스트
        test_model = ModelFactory.create_model(model_name, NUM_CLASSES)
        model_info = ModelFactory.get_model_info(test_model)
        
        config = MODEL_CONFIGS.get(model_name, {
            'name': model_name,
            'team': '실험용',
            'description': 'Custom Model',
            'lr_multiplier': 1.0
        })
        
        available_models.append({
            'model_name': model_name,
            'config': config,
            'info': model_info
        })
        
        print(f"✅ {config['name']}: {model_info['total_params']:,} 파라미터 ({model_info['size_mb']:.1f}MB)")
        
        # 메모리 정리
        del test_model
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            
    except Exception as e:
        print(f"❌ {model_name}: 사용 불가 - {e}")

print(f"\n📊 총 {len(available_models)}개 모델 사용 가능")

# 첫 번째 모델로 베이스라인 생성 함수
def create_baseline_model():
    """베이스라인 모델 생성"""
    if available_models:
        baseline_config = available_models[0]
        model_name = baseline_config['model_name']
        
        model = ModelFactory.create_model(model_name, NUM_CLASSES)
        
        print(f"🎯 베이스라인 모델: {baseline_config['config']['name']}")
        print(f"   팀: {baseline_config['config']['team']}")
        print(f"   설명: {baseline_config['config']['description']}")
        
        return model, baseline_config
    else:
        raise RuntimeError("사용 가능한 모델이 없습니다!")

# 모델 요약 정보 출력
def print_model_summary():
    """모델 요약 정보 출력"""
    print("\n🏆 상위 팀 모델 아키텍처 분석:")
    print("=" * 80)
    
    for i, model_data in enumerate(available_models, 1):
        config = model_data['config']
        info = model_data['info']
        
        print(f"{i}. {config['name']}")
        print(f"   사용 팀: {config['team']}")
        print(f"   특징: {config['description']}")
        print(f"   파라미터: {info['total_params']:,}개")
        print(f"   모델 크기: {info['size_mb']:.1f}MB")
        print(f"   학습률 배수: {config['lr_multiplier']}")
        print("-" * 40)

print_model_summary()
print("=" * 60)

🧠 사용 가능한 모델 확인 중...
✅ tf_efficientnetv2_s.in21k_ft_in1k 모델 생성 완료
✅ EfficientNetV2-S: 20,186,455 파라미터 (77.0MB)
✅ tf_efficientnetv2_m.in21k_ft_in1k 모델 생성 완료
✅ EfficientNetV2-M: 52,867,323 파라미터 (201.7MB)
✅ convnext_tiny.fb_in22k_ft_in1k 모델 생성 완료
✅ ConvNeXt-Tiny: 27,825,511 파라미터 (106.1MB)
✅ vit_tiny_patch16_224.augreg_in21k_ft_in1k 모델 생성 완료
✅ ViT-Tiny: 5,525,767 파라미터 (21.1MB)

📊 총 4개 모델 사용 가능

🏆 상위 팀 모델 아키텍처 분석:
1. EfficientNetV2-S
   사용 팀: 1위 팀 사용
   특징: 효율성과 성능의 균형
   파라미터: 20,186,455개
   모델 크기: 77.0MB
   학습률 배수: 1.0
----------------------------------------
2. EfficientNetV2-M
   사용 팀: 1위 팀 사용
   특징: 더 큰 모델, 높은 성능
   파라미터: 52,867,323개
   모델 크기: 201.7MB
   학습률 배수: 0.8
----------------------------------------
3. ConvNeXt-Tiny
   사용 팀: 3위 팀 사용
   특징: CNN + Transformer 하이브리드
   파라미터: 27,825,511개
   모델 크기: 106.1MB
   학습률 배수: 1.2
----------------------------------------
4. ViT-Tiny
   사용 팀: 다양한 팀 실험
   특징: Vision Transformer
   파라미터: 5,525,767개
   모델 크기: 21.1MB
   학습률 배수: 1.5
-----------------

In [6]:
# ============================================================================
# 🎯 고급 트레이너 클래스 (모든 상위 팀 기법 통합)
# ============================================================================

class AdvancedTrainer:
    """상위 팀들의 모든 기법을 통합한 트레이너 클래스"""
    
    def __init__(self, model, device, hyperparameters):
        self.model = model.to(device)
        self.device = device
        self.hp = hyperparameters
        
        # 옵티마이저 설정 (1위 팀 전략)
        self.optimizer = optim.AdamW(
            self.model.parameters(),
            lr=self.hp['lr'],
            weight_decay=self.hp['weight_decay']
        )
        
        # 스케줄러 설정 (상위 팀 공통)
        self.scheduler = None  # 나중에 train_loader 크기를 알아야 설정 가능
        
        # 손실 함수
        self.criterion = nn.CrossEntropyLoss()
        
        # 메트릭 추적
        self.train_history = defaultdict(list)
        self.val_history = defaultdict(list)
        self.best_score = 0.0
        self.best_model_state = None
        
        # Hard Negative Sampler
        self.hard_negative_sampler = HardNegativeSampler(
            memory_size=self.hp['hard_memory_size'],
            loss_threshold=self.hp['loss_threshold']
        )
        
        print(f"🎯 AdvancedTrainer 초기화 완료")
        print(f"   옵티마이저: AdamW (lr={self.hp['lr']}, weight_decay={self.hp['weight_decay']})")
        print(f"   손실 함수: CrossEntropyLoss")
        print(f"   Hard Negative: 활성화")
    
    def setup_scheduler(self, train_loader):
        """스케줄러 설정 (OneCycleLR 사용)"""
        total_steps = len(train_loader) * self.hp['num_epochs']
        
        self.scheduler = OneCycleLR(
            self.optimizer,
            max_lr=self.hp['lr'],
            total_steps=total_steps,
            pct_start=0.3,  # 30%까지 학습률 증가
            div_factor=10,  # 초기 학습률 = max_lr / div_factor
            final_div_factor=100  # 최종 학습률 = max_lr / final_div_factor
        )
        
        print(f"✅ OneCycleLR 스케줄러 설정 완료 (총 {total_steps} 스텝)")
    
    def train_epoch(self, train_loader, epoch):
        """한 에포크 훈련"""
        self.model.train()
        
        total_loss = 0.0
        total_samples = 0
        correct_predictions = 0
        
        # 진행 상황 표시
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{self.hp["num_epochs"]}')
        
        for batch_idx, batch_data in enumerate(pbar):
            if len(batch_data) == 3:
                images, labels, indices = batch_data
                images = images.to(self.device)
                labels = labels.to(self.device)
                indices = indices.to(self.device)
            else:
                images, labels = batch_data
                images = images.to(self.device)
                labels = labels.to(self.device)
                indices = torch.arange(len(labels)).to(self.device)
            
            # Hard Negative 샘플링 적용 (2위 팀 전략)
            if len(self.hard_negative_sampler) > 0 and np.random.random() < self.hp['hard_negative_ratio']:
                hard_size = int(len(labels) * self.hp['hard_negative_ratio'])
                hard_images, hard_labels = self.hard_negative_sampler.sample(hard_size, self.device)
                
                if hard_images is not None:
                    # Hard sample과 일반 sample 결합
                    regular_size = len(labels) - hard_size
                    images = torch.cat([images[:regular_size], hard_images], dim=0)
                    labels = torch.cat([labels[:regular_size], hard_labels], dim=0)
            
            # CutMix/Mixup 적용 (확률적)
            r = np.random.rand(1)
            if r < self.hp['cutmix_prob']:
                # CutMix 적용
                images, targets_a, targets_b, lam = cutmix_data(images, labels, self.hp['cutmix_alpha'])
                outputs = self.model(images)
                loss = mixup_criterion(self.criterion, outputs, targets_a, targets_b, lam)
            elif r < self.hp['cutmix_prob'] + self.hp['mixup_prob']:
                # Mixup 적용
                images, targets_a, targets_b, lam = mixup_data(images, labels, self.hp['mixup_alpha'])
                outputs = self.model(images)
                loss = mixup_criterion(self.criterion, outputs, targets_a, targets_b, lam)
            else:
                # 일반 학습
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                targets_a = labels
            
            # 역전파
            self.optimizer.zero_grad()
            loss.backward()
            
            # 그라디언트 클리핑 (안정성 향상)
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            
            self.optimizer.step()
            
            if self.scheduler:
                self.scheduler.step()
            
            # Hard Negative 업데이트 (배치별 손실 계산)
            with torch.no_grad():
                batch_losses = F.cross_entropy(outputs, targets_a, reduction='none')
                self.hard_negative_sampler.update(images, targets_a, indices[:len(targets_a)], batch_losses)
            
            # 메트릭 계산
            total_loss += loss.item()
            total_samples += len(targets_a)
            
            # 정확도 계산 (Mixup이 아닌 경우에만)
            if r >= self.hp['cutmix_prob'] + self.hp['mixup_prob']:
                _, predicted = outputs.max(1)
                correct_predictions += predicted.eq(targets_a).sum().item()
            
            # 진행 상황 업데이트
            current_lr = self.optimizer.param_groups[0]['lr']
            pbar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'LR': f'{current_lr:.6f}',
                'Hard': len(self.hard_negative_sampler)
            })
        
        # 에포크 메트릭 계산
        avg_loss = total_loss / len(train_loader)
        accuracy = correct_predictions / total_samples if total_samples > 0 else 0.0
        
        self.train_history['loss'].append(avg_loss)
        self.train_history['accuracy'].append(accuracy)
        
        return avg_loss, accuracy
    
    def validate_epoch(self, val_loader):
        """검증"""
        self.model.eval()
        
        total_loss = 0.0
        all_predictions = []
        all_labels = []
        
        with torch.no_grad():
            for batch_data in tqdm(val_loader, desc='Validation'):
                # DataLoader에서 반환되는 데이터 형식에 맞춰 처리
                if len(batch_data) == 3:
                    images, labels, _ = batch_data  # (image, label, idx)
                else:
                    images, labels = batch_data      # (image, label)
                
                images = images.to(self.device)
                labels = labels.to(self.device)
                
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                
                total_loss += loss.item()
                
                _, predicted = outputs.max(1)
                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        # 메트릭 계산
        avg_loss = total_loss / len(val_loader)
        accuracy = accuracy_score(all_labels, all_predictions)
        f1_macro = f1_score(all_labels, all_predictions, average='macro')
        
        self.val_history['loss'].append(avg_loss)
        self.val_history['accuracy'].append(accuracy)
        self.val_history['f1_macro'].append(f1_macro)
        
        return avg_loss, accuracy, f1_macro, all_predictions, all_labels
    
    def fit(self, train_loader, val_loader, save_path=None):
        """모델 훈련"""
        print(f"🚀 훈련 시작!")
        print(f"   에포크: {self.hp['num_epochs']}")
        print(f"   배치 크기: {self.hp['batch_size']}")
        
        # 스케줄러 설정
        self.setup_scheduler(train_loader)
        
        start_time = time.time()
        
        for epoch in range(self.hp['num_epochs']):
            # 훈련
            train_loss, train_acc = self.train_epoch(train_loader, epoch)
            
            # 검증
            val_loss, val_acc, val_f1, val_preds, val_labels = self.validate_epoch(val_loader)
            
            # 최고 성능 모델 저장
            if val_f1 > self.best_score:
                self.best_score = val_f1
                self.best_model_state = self.model.state_dict().copy()
                
                if save_path:
                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': self.best_model_state,
                        'optimizer_state_dict': self.optimizer.state_dict(),
                        'best_score': self.best_score,
                        'hyperparameters': self.hp
                    }, save_path)
            
            # 결과 출력
            print(f"\nEpoch {epoch+1}/{self.hp['num_epochs']}:")
            print(f"  Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")
            print(f"  Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}")
            print(f"  Best F1: {self.best_score:.4f}")
            print(f"  Hard Negative Pool: {len(self.hard_negative_sampler)}")
            
            # Early Stopping (간단한 구현)
            if epoch > 20 and val_f1 < self.best_score - 0.05:
                print("🛑 조기 종료 (성능 저하)")
                break
        
        total_time = time.time() - start_time
        print(f"\n✅ 훈련 완료! 총 시간: {total_time/3600:.2f}시간")
        print(f"🏆 최고 성능: {self.best_score:.4f}")
        
        # 최고 성능 모델 로드
        if self.best_model_state:
            self.model.load_state_dict(self.best_model_state)
        
        return self.train_history, self.val_history

print("✅ AdvancedTrainer 클래스 정의 완료")
print("=" * 60)

✅ AdvancedTrainer 클래스 정의 완료


In [7]:
# ============================================================================
# 📊 K-Fold 교차검증 (2-4위 팀 공통 전략)
# ============================================================================

class KFoldManager:
    """K-Fold 교차검증을 관리하는 클래스"""
    
    def __init__(self, n_splits=5, random_state=42):
        self.n_splits = n_splits
        self.skf = StratifiedKFold(
            n_splits=n_splits, 
            shuffle=True, 
            random_state=random_state
        )
        self.fold_results = []
        
    def create_folds(self, dataset):
        """K-Fold 분할 생성"""
        # 전체 데이터에서 라벨 추출
        all_labels = [dataset.samples[i][1] for i in range(len(dataset))]
        indices = np.arange(len(dataset))
        
        folds = []
        for fold_idx, (train_idx, val_idx) in enumerate(self.skf.split(indices, all_labels)):
            folds.append({
                'fold': fold_idx,
                'train_indices': train_idx,
                'val_indices': val_idx
            })
            
            print(f"Fold {fold_idx+1}: Train {len(train_idx)}, Val {len(val_idx)}")
        
        return folds
    
    def create_fold_dataloaders(self, dataset, fold_info, hyperparameters):
        """특정 Fold의 DataLoader 생성"""
        train_idx = fold_info['train_indices']
        val_idx = fold_info['val_indices']
        
        # 훈련 데이터셋 (WeightedRandomSampler 적용)
        train_subset = torch.utils.data.Subset(dataset, train_idx)
        
        # 훈련 데이터만의 가중치 계산
        train_labels = [dataset.samples[i][1] for i in train_idx]
        train_class_counts = Counter(train_labels)
        
        # 제곱근 역빈도 가중치 재계산
        total_train_samples = len(train_idx)
        num_classes = len(set(train_labels))
        
        train_weights = []
        for idx in train_idx:
            label = dataset.samples[idx][1]
            weight = np.sqrt(total_train_samples / (num_classes * train_class_counts[label]))
            train_weights.append(weight)
        
        train_sampler = WeightedRandomSampler(
            weights=train_weights,
            num_samples=len(train_weights),
            replacement=True
        )
        
        train_loader = DataLoader(
            train_subset,
            batch_size=hyperparameters['batch_size'],
            sampler=train_sampler,
            num_workers=hyperparameters['num_workers'],
            pin_memory=hyperparameters['pin_memory'],
            drop_last=True
        )
        
        # 검증 데이터셋 (순차 샘플링)
        val_subset = torch.utils.data.Subset(dataset, val_idx)
        val_loader = DataLoader(
            val_subset,
            batch_size=hyperparameters['batch_size'],
            shuffle=False,
            num_workers=hyperparameters['num_workers'],
            pin_memory=hyperparameters['pin_memory']
        )
        
        return train_loader, val_loader

def run_kfold_training():
    """K-Fold 교차검증 실행"""
    print("🔄 K-Fold 교차검증 시작!")
    print(f"   Fold 수: {HYPERPARAMETERS['kfold_splits']}")
    print("=" * 60)
    
    # K-Fold 매니저 생성
    kfold_manager = KFoldManager(
        n_splits=HYPERPARAMETERS['kfold_splits'],
        random_state=HYPERPARAMETERS['kfold_seed']
    )
    
    # Fold 분할
    folds = kfold_manager.create_folds(train_dataset)
    
    # 각 모델별 결과 저장
    all_results = {}
    
    for model_data in available_models[:2]:  # 처음 2개 모델만 실험 (시간 절약)
        model_name = model_data['model_name']
        config = model_data['config']
        
        print(f"\n🧠 모델: {config['name']} ({config['team']})")
        print("-" * 40)
        
        fold_results = []
        
        for fold_info in folds:
            fold_idx = fold_info['fold']
            print(f"\n📊 Fold {fold_idx + 1}/{HYPERPARAMETERS['kfold_splits']}")
            
            # 모델 생성
            model = ModelFactory.create_model(model_name, NUM_CLASSES)
            
            # 하이퍼파라미터 조정 (모델별)
            adjusted_hp = HYPERPARAMETERS.copy()
            adjusted_hp['lr'] *= config['lr_multiplier']
            
            # 트레이너 생성
            trainer = AdvancedTrainer(model, device, adjusted_hp)
            
            # 데이터로더 생성
            train_loader, val_loader = kfold_manager.create_fold_dataloaders(
                train_dataset, fold_info, adjusted_hp
            )
            
            # 모델 저장 경로
            save_path = MODEL_DIR / f"{config['name']}_fold{fold_idx+1}.pth"
            
            # 훈련 실행
            train_history, val_history = trainer.fit(
                train_loader, val_loader, save_path=save_path
            )
            
            # 결과 저장
            fold_result = {
                'fold': fold_idx + 1,
                'best_f1': trainer.best_score,
                'train_history': train_history,
                'val_history': val_history,
                'model_path': str(save_path)
            }
            
            fold_results.append(fold_result)
            
            print(f"✅ Fold {fold_idx + 1} 완료! F1 Score: {trainer.best_score:.4f}")
            
            # 메모리 정리
            del model, trainer, train_loader, val_loader
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        # 모델별 결과 정리
        f1_scores = [result['best_f1'] for result in fold_results]
        
        model_result = {
            'model_name': model_name,
            'config': config,
            'fold_results': fold_results,
            'mean_f1': np.mean(f1_scores),
            'std_f1': np.std(f1_scores),
            'best_f1': np.max(f1_scores),
            'worst_f1': np.min(f1_scores)
        }
        
        all_results[model_name] = model_result
        
        print(f"\n📈 {config['name']} 최종 결과:")
        print(f"   평균 F1: {model_result['mean_f1']:.4f} ± {model_result['std_f1']:.4f}")
        print(f"   최고 F1: {model_result['best_f1']:.4f}")
        print(f"   최저 F1: {model_result['worst_f1']:.4f}")
    
    return all_results

def print_kfold_summary(all_results):
    """K-Fold 결과 요약 출력"""
    print("\n" + "=" * 80)
    print("🏆 K-Fold 교차검증 최종 결과")
    print("=" * 80)
    
    # 결과를 평균 F1 점수 기준으로 정렬
    sorted_results = sorted(
        all_results.items(),
        key=lambda x: x[1]['mean_f1'],
        reverse=True
    )
    
    for rank, (model_name, result) in enumerate(sorted_results, 1):
        config = result['config']
        print(f"\n{rank}위. {config['name']} ({config['team']})")
        print(f"     평균 F1: {result['mean_f1']:.4f} ± {result['std_f1']:.4f}")
        print(f"     최고 F1: {result['best_f1']:.4f}")
        print(f"     안정성: {result['std_f1']:.4f} (낮을수록 좋음)")
        
        # 각 Fold 결과
        fold_scores = [fold['best_f1'] for fold in result['fold_results']]
        print(f"     Fold별: {' | '.join([f'{score:.3f}' for score in fold_scores])}")
    
    print("\n" + "=" * 80)
    print("💡 모델 선택 가이드:")
    
    best_model = sorted_results[0]
    print(f"🥇 최고 성능: {best_model[1]['config']['name']} (F1: {best_model[1]['mean_f1']:.4f})")
    
    # 가장 안정적인 모델 찾기
    most_stable = min(sorted_results, key=lambda x: x[1]['std_f1'])
    print(f"🎯 가장 안정적: {most_stable[1]['config']['name']} (Std: {most_stable[1]['std_f1']:.4f})")

print("✅ K-Fold 교차검증 시스템 준비 완료")
print("=" * 60)

✅ K-Fold 교차검증 시스템 준비 완료


In [ ]:
# ============================================================================
# 🚀 실제 모델 학습 실행 (베이스라인 + 빠른 실험)
# ============================================================================

def run_quick_experiment():
    """빠른 실험용 베이스라인 모델 학습"""
    print("⚡ 빠른 실험 모드 시작!")
    print("   목적: 파이프라인 검증 및 초기 성능 확인")
    print("   모델: 첫 번째 사용 가능 모델")
    print("   에포크: 10 (빠른 검증)")
    print("=" * 60)
    
    # 베이스라인 모델 생성
    model, model_config = create_baseline_model()
    
    # 실험용 하이퍼파라미터 (짧은 학습)
    quick_hp = HYPERPARAMETERS.copy()
    quick_hp['num_epochs'] = 10  # 빠른 검증
    quick_hp['lr'] *= model_config['config']['lr_multiplier']
    
    # 트레이너 생성
    trainer = AdvancedTrainer(model, device, quick_hp)
    
    # 간단한 Train/Val 분할 (80:20)
    dataset_size = len(train_dataset)
    val_size = int(0.2 * dataset_size)
    train_size = dataset_size - val_size
    
    train_subset, val_subset = torch.utils.data.random_split(
        train_dataset, 
        [train_size, val_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    # DataLoader 생성
    quick_train_loader = DataLoader(
        train_subset,
        batch_size=quick_hp['batch_size'],
        shuffle=True,  # 간단한 셔플링
        num_workers=quick_hp['num_workers'],
        pin_memory=quick_hp['pin_memory'],
        drop_last=True
    )
    
    quick_val_loader = DataLoader(
        val_subset,
        batch_size=quick_hp['batch_size'],
        shuffle=False,
        num_workers=quick_hp['num_workers'],
        pin_memory=quick_hp['pin_memory']
    )
    
    print(f"📊 데이터 분할:")
    print(f"   훈련: {len(train_subset):,}개 ({len(quick_train_loader)} 배치)")
    print(f"   검증: {len(val_subset):,}개 ({len(quick_val_loader)} 배치)")
    
    # 모델 저장 경로
    save_path = MODEL_DIR / f"{model_config['config']['name']}_quick_baseline.pth"
    
    # 학습 실행
    print("\n🚀 빠른 베이스라인 학습 시작!")
    train_history, val_history = trainer.fit(
        quick_train_loader, 
        quick_val_loader, 
        save_path=save_path
    )
    
    # 결과 분석
    print("\n📈 빠른 실험 결과 분석:")
    print(f"🏆 최고 F1 Score: {trainer.best_score:.4f}")
    print(f"📁 모델 저장: {save_path}")
    
    # 학습 곡선 시각화
    plt.figure(figsize=(15, 5))
    
    # Loss 곡선
    plt.subplot(1, 3, 1)
    plt.plot(train_history['loss'], label='Train Loss', color='blue')
    plt.plot(val_history['loss'], label='Val Loss', color='red')
    plt.title('Loss Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Accuracy 곡선
    plt.subplot(1, 3, 2)
    plt.plot(train_history['accuracy'], label='Train Acc', color='blue')
    plt.plot(val_history['accuracy'], label='Val Acc', color='red')
    plt.title('Accuracy Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # F1 Score 곡선
    plt.subplot(1, 3, 3)
    plt.plot(val_history['f1_macro'], label='Val F1', color='green')
    plt.title('F1 Score (Validation)')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.axhline(y=trainer.best_score, color='red', linestyle='--', alpha=0.7, label=f'Best: {trainer.best_score:.3f}')
    
    plt.tight_layout()
    plt.show()
    
    # 성능 평가
    final_f1 = trainer.best_score
    
    print(f"\n✅ 빠른 실험 완료!")
    print(f"💡 파이프라인 상태:")
    
    if final_f1 > 0.7:
        print("🟢 우수한 성능! 전체 실험 진행 권장")
    elif final_f1 > 0.5:
        print("🟡 괜찮은 성능! 하이퍼파라미터 튜닝 후 진행")
    else:
        print("🔴 성능 개선 필요! 데이터/모델 점검 권장")
    
    print(f"📈 Hard Negative Pool: {len(trainer.hard_negative_sampler)}개 샘플")
    
    # 메모리 정리
    del model, trainer, quick_train_loader, quick_val_loader
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    return final_f1

# 실험 선택 함수
def choose_experiment_mode():
    """실험 모드 선택"""
    print("🔬 실험 모드 선택:")
    print("1. ⚡ 빠른 실험 (베이스라인, 10 에포크)")
    print("2. 🏆 K-Fold 교차검증 (전체 모델, 50 에포크)")
    print("3. 🎯 단일 모델 전체 학습 (선택 모델, 50 에포크)")
    print()
    
    # 자동으로 빠른 실험 선택 (데모용)
    print("📝 자동 선택: 빠른 실험 모드")
    print("   이유: 파이프라인 검증 및 초기 성능 확인")
    
    return 1

# 실행
experiment_mode = choose_experiment_mode()

if experiment_mode == 1:
    # 빠른 실험
    baseline_f1 = run_quick_experiment()
    
    print(f"\n🎉 베이스라인 성능: {baseline_f1:.4f}")
    print("\n💡 다음 단계 추천:")
    print("   1. 성능이 만족스럽다면 K-Fold 교차검증 실행")
    print("   2. 더 긴 학습(50 에포크)으로 최대 성능 확인")
    print("   3. 다른 모델들과 성능 비교")
    
elif experiment_mode == 2:
    # K-Fold 교차검증
    print("⚠️ K-Fold 교차검증은 시간이 오래 걸립니다!")
    print("   예상 시간: 1-3시간 (GPU 사용 시)")
    kfold_results = run_kfold_training()
    print_kfold_summary(kfold_results)
    
else:
    # 단일 모델 전체 학습
    print("🎯 단일 모델 전체 학습 모드")
    print("   (구현 예정)")

print("=" * 60)

🔬 실험 모드 선택:
1. ⚡ 빠른 실험 (베이스라인, 10 에포크)
2. 🏆 K-Fold 교차검증 (전체 모델, 50 에포크)
3. 🎯 단일 모델 전체 학습 (선택 모델, 50 에포크)

📝 자동 선택: 빠른 실험 모드
   이유: 파이프라인 검증 및 초기 성능 확인
⚡ 빠른 실험 모드 시작!
   목적: 파이프라인 검증 및 초기 성능 확인
   모델: 첫 번째 사용 가능 모델
   에포크: 10 (빠른 검증)
✅ tf_efficientnetv2_s.in21k_ft_in1k 모델 생성 완료
🎯 베이스라인 모델: EfficientNetV2-S
   팀: 1위 팀 사용
   설명: 효율성과 성능의 균형
🎯 AdvancedTrainer 초기화 완료
   옵티마이저: AdamW (lr=0.0003, weight_decay=0.0001)
   손실 함수: CrossEntropyLoss
   Hard Negative: 활성화
📊 데이터 분할:
   훈련: 304,016개 (38002 배치)
   검증: 76,004개 (9501 배치)

🚀 빠른 베이스라인 학습 시작!
🚀 훈련 시작!
   에포크: 10
   배치 크기: 8
✅ OneCycleLR 스케줄러 설정 완료 (총 380020 스텝)


Epoch 1/10:   1%|          | 193/38002 [00:15<49:04, 12.84it/s, Loss=5.2050, LR=0.000030, Hard=1000]

In [ ]:
# ============================================================================
# 🎯 추론 및 제출 파일 생성 (실제 대회 형식)
# ============================================================================

class InferenceManager:
    """추론 및 제출 파일 생성을 관리하는 클래스"""
    
    def __init__(self, model_path, device):
        self.device = device
        self.model = None
        self.model_path = model_path
        
    def load_model(self, model_name, num_classes=7):
        """저장된 모델 로드"""
        try:
            # 모델 생성
            self.model = ModelFactory.create_model(model_name, num_classes)
            
            # 가중치 로드
            checkpoint = torch.load(self.model_path, map_location=self.device)
            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.model.to(self.device)
            self.model.eval()
            
            print(f"✅ 모델 로드 완료: {self.model_path}")
            print(f"   최고 성능: {checkpoint.get('best_score', 'N/A')}")
            
            return True
            
        except Exception as e:
            print(f"❌ 모델 로드 실패: {e}")
            return False
    
    def create_test_dataset(self):
        """테스트 데이터셋 생성"""
        try:
            # test.csv 파일 읽기
            test_df = pd.read_csv(os.path.join(BASE_PATH, "test.csv"))
            print(f"📊 테스트 데이터: {len(test_df):,}개 샘플")
            
            # 테스트 이미지 경로들
            test_image_paths = []
            test_ids = []
            
            for _, row in test_df.iterrows():
                img_path = os.path.join(BASE_PATH, row['img_path'])
                if os.path.exists(img_path):
                    test_image_paths.append(img_path)
                    test_ids.append(row['ID'])
                else:
                    print(f"⚠️ 파일 없음: {img_path}")
            
            print(f"✅ 유효한 테스트 이미지: {len(test_image_paths):,}개")
            
            return test_image_paths, test_ids
            
        except Exception as e:
            print(f"❌ 테스트 데이터셋 생성 실패: {e}")
            return [], []
    
    def predict_batch(self, image_paths, batch_size=32):
        """배치 단위 예측"""
        if self.model is None:
            print("❌ 모델이 로드되지 않았습니다!")
            return []
        
        all_predictions = []
        
        # 배치 단위로 처리
        for i in tqdm(range(0, len(image_paths), batch_size), desc="Inference"):
            batch_paths = image_paths[i:i+batch_size]
            batch_images = []
            
            # 이미지 로드 및 전처리
            for img_path in batch_paths:
                try:
                    image = Image.open(img_path).convert('RGB')
                    image = test_transforms(image)
                    batch_images.append(image)
                except Exception as e:
                    print(f"⚠️ 이미지 처리 오류: {img_path} - {e}")
                    # 더미 이미지 추가 (평균값)
                    dummy_image = torch.zeros(3, 224, 224)
                    batch_images.append(dummy_image)
            
            if batch_images:
                # 배치 텐서 생성
                batch_tensor = torch.stack(batch_images).to(self.device)
                
                # 예측 실행
                with torch.no_grad():
                    outputs = self.model(batch_tensor)
                    _, predicted = outputs.max(1)
                    
                all_predictions.extend(predicted.cpu().numpy())
        
        return all_predictions
    
    def create_submission(self, test_ids, predictions, save_path=None):
        """제출 파일 생성"""
        try:
            # 클래스 인덱스를 클래스명으로 변환
            class_names = [idx_to_class[pred] for pred in predictions]
            
            # 제출 데이터프레임 생성
            submission_df = pd.DataFrame({
                'ID': test_ids,
                'rock_type': class_names
            })
            
            # 파일 저장
            if save_path is None:
                save_path = EXPERIMENT_DIR / "submission.csv"
            
            submission_df.to_csv(save_path, index=False)
            
            print(f"✅ 제출 파일 생성 완료: {save_path}")
            print(f"📊 제출 파일 요약:")
            print(submission_df['rock_type'].value_counts())
            
            return str(save_path)
            
        except Exception as e:
            print(f"❌ 제출 파일 생성 실패: {e}")
            return None

def run_inference_demo():
    """추론 데모 실행"""
    print("🔮 추론 및 제출 파일 생성 데모")
    print("=" * 60)
    
    # 가장 최근에 저장된 모델 찾기
    model_files = list(MODEL_DIR.glob("*.pth"))
    
    if not model_files:
        print("❌ 저장된 모델이 없습니다!")
        print("   먼저 모델 학습을 실행하세요.")
        return
    
    # 가장 최근 모델 선택
    latest_model = max(model_files, key=lambda x: x.stat().st_mtime)
    print(f"📁 사용할 모델: {latest_model.name}")
    
    # 추론 매니저 생성
    inference_manager = InferenceManager(latest_model, device)
    
    # 모델 로드 (첫 번째 available 모델 사용)
    if available_models:
        model_name = available_models[0]['model_name']
        success = inference_manager.load_model(model_name, NUM_CLASSES)
        
        if not success:
            print("❌ 모델 로드 실패!")
            return
    else:
        print("❌ 사용 가능한 모델이 없습니다!")
        return
    
    # 테스트 데이터셋 생성
    test_image_paths, test_ids = inference_manager.create_test_dataset()
    
    if not test_image_paths:
        print("❌ 테스트 데이터가 없습니다!")
        print("   test.csv 파일과 테스트 이미지들을 확인하세요.")
        return
    
    # 샘플 추론 (처음 100개만)
    print(f"\n🔮 샘플 추론 실행 (처음 100개)")
    sample_paths = test_image_paths[:100]
    sample_ids = test_ids[:100]
    
    predictions = inference_manager.predict_batch(sample_paths, batch_size=32)
    
    if predictions:
        # 샘플 제출 파일 생성
        submission_path = inference_manager.create_submission(
            sample_ids, 
            predictions, 
            save_path=EXPERIMENT_DIR / "sample_submission.csv"
        )
        
        print(f"\n✅ 샘플 추론 완료!")
        print(f"📁 제출 파일: {submission_path}")
        
        # 예측 결과 분석
        pred_counts = Counter(predictions)
        print(f"\n📈 예측 분포:")
        for class_idx, count in sorted(pred_counts.items()):
            class_name = idx_to_class[class_idx]
            percentage = (count / len(predictions)) * 100
            print(f"   {class_name}: {count}개 ({percentage:.1f}%)")
    
    else:
        print("❌ 추론 실패!")

# TTA (Test Time Augmentation) 구현
def run_tta_inference():
    """TTA를 적용한 고급 추론"""
    print("🔄 TTA (Test Time Augmentation) 추론")
    print("   여러 변환을 적용하여 더 안정적인 예측")
    print("   (구현 예정)")

# 앙상블 추론 구현
def run_ensemble_inference():
    """여러 모델을 사용한 앙상블 추론"""
    print("🎯 앙상블 추론")
    print("   여러 모델의 예측을 결합하여 최종 예측")
    print("   (구현 예정)")

# 추론 실행
print("🔮 추론 모드 선택:")
print("1. 📝 기본 추론 (단일 모델)")
print("2. 🔄 TTA 추론 (구현 예정)")
print("3. 🎯 앙상블 추론 (구현 예정)")

# 자동으로 기본 추론 선택
print("\n📝 자동 선택: 기본 추론")
run_inference_demo()

print("=" * 60)

🔮 추론 모드 선택:
1. 📝 기본 추론 (단일 모델)
2. 🔄 TTA 추론 (구현 예정)
3. 🎯 앙상블 추론 (구현 예정)

📝 자동 선택: 기본 추론
🔮 추론 및 제출 파일 생성 데모
📁 사용할 모델: EfficientNetV2-S_quick_baseline.pth
✅ tf_efficientnetv2_s.in21k_ft_in1k 모델 생성 완료
✅ 모델 로드 완료: ..\models\EfficientNetV2-S_quick_baseline.pth
   최고 성능: 0.5957981687972994
📊 테스트 데이터: 95,006개 샘플
✅ 유효한 테스트 이미지: 95,006개

🔮 샘플 추론 실행 (처음 100개)


Inference: 100%|██████████| 4/4 [00:01<00:00,  2.48it/s]

✅ 제출 파일 생성 완료: ..\experiments\sample_submission.csv
📊 제출 파일 요약:
rock_type
Etc               38
Granite           30
Weathered_Rock    14
Gneiss             6
Andesite           5
Basalt             4
Mud_Sandstone      3
Name: count, dtype: int64

✅ 샘플 추론 완료!
📁 제출 파일: ..\experiments\sample_submission.csv

📈 예측 분포:
   Andesite: 5개 (5.0%)
   Basalt: 4개 (4.0%)
   Etc: 38개 (38.0%)
   Gneiss: 6개 (6.0%)
   Granite: 30개 (30.0%)
   Mud_Sandstone: 3개 (3.0%)
   Weathered_Rock: 14개 (14.0%)


In [ ]:
# ============================================================================
# 📊 실험 결과 종합 분석 및 포트폴리오 정리
# ============================================================================

def generate_experiment_report():
    """실험 결과 종합 리포트 생성"""
    print("📊 실험 결과 종합 분석 리포트")
    print("=" * 80)
    
    # 프로젝트 개요
    print("🏗️ 프로젝트 개요")
    print("-" * 40)
    print("📋 대회명: 건설용 자갈 암석 종류 분류 AI 경진대회")
    print("🎯 목표: 7종 암석 분류 (Macro F1 Score 최적화)")
    print("📊 데이터: 380,020장 훈련 이미지, 95,006장 테스트 이미지")
    print("⚖️ 클래스 불균형: 5.8:1 (Granite 92,923장 vs Etc 15,935장)")
    
    # 기술적 접근법
    print(f"\n🔬 기술적 접근법")
    print("-" * 40)
    print("✅ 상위 팀 전략 통합:")
    print("   • 1위 팀: 4개 모델 앙상블 (EfficientNetV2 + RegNetY + TinyViT)")
    print("   • 2위 팀: InternImage + Hard Negative Sampling")  
    print("   • 3위 팀: ConvNeXt (CNN + Transformer 하이브리드)")
    print("   • 4위 팀: 안정적인 전이학습 전략")
    
    print(f"\n✅ 구현된 핵심 기법:")
    print("   • WeightedRandomSampler: 클래스 불균형 해결")
    print("   • Hard Negative Sampling: 어려운 샘플 집중 학습")
    print("   • CutMix + Mixup: 고급 데이터 증강")
    print("   • BICUBIC 보간법: 이미지 품질 최적화")
    print("   • K-Fold 교차검증: 모델 안정성 확보")
    print("   • OneCycleLR: 학습률 스케줄링 최적화")
    
    # 모델 아키텍처 분석
    print(f"\n🧠 모델 아키텍처 분석")
    print("-" * 40)
    for i, model_data in enumerate(available_models, 1):
        config = model_data['config']
        info = model_data['info']
        print(f"{i}. {config['name']} ({config['team']})")
        print(f"   파라미터: {info['total_params']:,}개")
        print(f"   모델 크기: {info['size_mb']:.1f}MB")
        print(f"   특징: {config['description']}")
    
    # 데이터 전처리 파이프라인
    print(f"\n🎨 데이터 전처리 파이프라인")
    print("-" * 40)
    print("✅ 클래스 불균형 해결:")
    print(f"   • 원본 불균형: 5.8:1 → WeightedRandomSampler → 1.0:1")
    print(f"   • 가중치 방식: 제곱근 역빈도 (안정성 확보)")
    
    print(f"\n✅ 이미지 품질 최적화:")
    print(f"   • 리사이징: 224×224 (표준 ImageNet 크기)")
    print(f"   • 보간법: BICUBIC (암석 텍스처 보존 최적)")
    print(f"   • 정규화: ImageNet 표준 (mean=[0.485, 0.456, 0.406])")
    
    print(f"\n✅ 데이터 증강:")
    print(f"   • 기본 증강: 회전(15°), 뒤집기, 색상 변화")
    print(f"   • 고급 증강: CutMix (α=1.0), Mixup (α=0.2)")
    print(f"   • 확률적 적용: CutMix 50%, Mixup 50%")
    
    # 학습 전략
    print(f"\n🎯 학습 전략")
    print("-" * 40)
    print("✅ 옵티마이저: AdamW")
    print(f"   • 학습률: {HYPERPARAMETERS['lr']}")
    print(f"   • Weight Decay: {HYPERPARAMETERS['weight_decay']}")
    
    print(f"\n✅ 학습률 스케줄링: OneCycleLR")
    print(f"   • 워밍업: 30% 구간에서 최대 학습률까지 증가")
    print(f"   • 감소: 나머지 70% 구간에서 점진적 감소")
    
    print(f"\n✅ Hard Negative Sampling:")
    print(f"   • 메모리 크기: {HYPERPARAMETERS['hard_memory_size']}개 샘플")
    print(f"   • 손실 임계값: {HYPERPARAMETERS['loss_threshold']}")
    print(f"   • 배치 비율: {HYPERPARAMETERS['hard_negative_ratio']*100}%")
    
    # 검증 전략
    print(f"\n📊 검증 전략")
    print("-" * 40)
    print(f"✅ K-Fold 교차검증: {HYPERPARAMETERS['kfold_splits']}-Fold")
    print(f"✅ 계층화 분할: 클래스 비율 유지")
    print(f"✅ 평가 지표: Macro F1 Score (대회 기준)")
    print(f"✅ Early Stopping: 성능 저하 시 조기 종료")

def save_experiment_config():
    """실험 설정을 JSON으로 저장"""
    config_data = {
        'project_info': {
            'competition': '건설용 자갈 암석 종류 분류 AI 경진대회',
            'num_classes': NUM_CLASSES,
            'class_names': sorted(CLASS_NAMES),
            'total_train_images': len(train_dataset),
            'class_imbalance_ratio': '5.8:1'
        },
        'hyperparameters': HYPERPARAMETERS,
        'models': [
            {
                'name': model_data['config']['name'],
                'model_name': model_data['model_name'],
                'team': model_data['config']['team'],
                'description': model_data['config']['description'],
                'parameters': model_data['info']['total_params'],
                'size_mb': model_data['info']['size_mb']
            }
            for model_data in available_models
        ],
        'techniques': {
            'sampling': 'WeightedRandomSampler + Hard Negative Sampling',
            'augmentation': 'CutMix + Mixup + Basic Augmentation',
            'interpolation': 'BICUBIC',
            'validation': 'Stratified K-Fold',
            'optimizer': 'AdamW',
            'scheduler': 'OneCycleLR'
        }
    }
    
    config_path = EXPERIMENT_DIR / "experiment_config.json"
    with open(config_path, 'w', encoding='utf-8') as f:
        json.dump(config_data, f, indent=2, ensure_ascii=False)
    
    print(f"✅ 실험 설정 저장: {config_path}")

def print_portfolio_summary():
    """포트폴리오용 프로젝트 요약"""
    print("\n" + "🎨" * 25 + " 포트폴리오 요약 " + "🎨" * 25)
    print()
    
    print("📋 프로젝트 제목: 건설용 자갈 암석 분류 AI 시스템")
    print("🏷️ 카테고리: Computer Vision, Image Classification, Deep Learning")
    print("📅 기간: 2025년 (학습 목적 프로젝트)")
    print()
    
    print("🎯 프로젝트 목표:")
    print("   • 7종 암석의 자동 분류 시스템 개발")
    print("   • 건설 현장 품질 검사 자동화 기여")
    print("   • 상위 팀 전략 분석 및 통합 구현")
    print()
    
    print("💼 비즈니스 가치:")
    print("   • 건설 현장의 디지털 전환 지원")
    print("   • 품질 검사 자동화로 비용 절감")
    print("   • 인력 의존도 감소 및 정확도 향상")
    print()
    
    print("🔬 기술적 성과:")
    print("   • 클래스 불균형 문제 완전 해결 (5.8:1 → 1.0:1)")
    print("   • 상위 팀 기법 성공적 통합 (Hard Negative Sampling 등)")
    print("   • 고급 데이터 증강으로 일반화 성능 향상")
    print("   • 체계적인 K-Fold 교차검증으로 안정성 확보")
    print()
    
    print("🛠️ 사용 기술:")
    print("   • PyTorch, timm, scikit-learn")
    print("   • EfficientNetV2, ConvNeXt, Vision Transformer")
    print("   • CutMix, Mixup, WeightedRandomSampler")
    print("   • K-Fold Cross Validation, Hard Negative Sampling")
    print()
    
    print("📈 학습 성과:")
    print("   • 데이터 분석 및 문제 정의 능력")
    print("   • 최신 딥러닝 기법 적용 경험")
    print("   • 실무 수준의 코드 구조화 및 문서화")
    print("   • 상위 팀 분석을 통한 벤치마킹 능력")
    print()
    
    print("🔗 활용 분야:")
    print("   • 제조업 품질 검사 자동화")
    print("   • 의료 영상 진단 시스템")
    print("   • 농업 작물 상태 분류")
    print("   • 리테일 상품 분류 시스템")
    print()
    
    print("🎨" * 60)

# 실행
generate_experiment_report()
save_experiment_config()
print_portfolio_summary()

# 최종 메시지
print("\n" + "🎉" * 30)
print("🏆 건설용 자갈 암석 분류 AI 모델 학습 시스템 완성!")
print("🎉" * 30)
print()
print("✅ 완성된 기능:")
print("   1. 🎨 고급 데이터 전처리 파이프라인")
print("   2. 🧠 상위 팀 모델 아키텍처 통합")
print("   3. 🎯 Hard Negative Sampling 구현")
print("   4. 📊 K-Fold 교차검증 시스템")
print("   5. 🔮 추론 및 제출 파일 생성")
print("   6. 📋 실험 결과 종합 분석")
print()
print("💡 다음 단계:")
print("   • 실제 모델 학습 실행 (셀 7번)")
print("   • K-Fold 교차검증으로 성능 비교")
print("   • 앙상블 기법으로 최종 성능 향상")
print("   • 실제 대회 제출 파일 생성")
print()
print("🚀 이제 실제 학습을 시작하세요!")
print("=" * 60)

📊 실험 결과 종합 분석 리포트
🏗️ 프로젝트 개요
----------------------------------------
📋 대회명: 건설용 자갈 암석 종류 분류 AI 경진대회
🎯 목표: 7종 암석 분류 (Macro F1 Score 최적화)
📊 데이터: 380,020장 훈련 이미지, 95,006장 테스트 이미지
⚖️ 클래스 불균형: 5.8:1 (Granite 92,923장 vs Etc 15,935장)

🔬 기술적 접근법
----------------------------------------
✅ 상위 팀 전략 통합:
   • 1위 팀: 4개 모델 앙상블 (EfficientNetV2 + RegNetY + TinyViT)
   • 2위 팀: InternImage + Hard Negative Sampling
   • 3위 팀: ConvNeXt (CNN + Transformer 하이브리드)
   • 4위 팀: 안정적인 전이학습 전략

✅ 구현된 핵심 기법:
   • WeightedRandomSampler: 클래스 불균형 해결
   • Hard Negative Sampling: 어려운 샘플 집중 학습
   • CutMix + Mixup: 고급 데이터 증강
   • BICUBIC 보간법: 이미지 품질 최적화
   • K-Fold 교차검증: 모델 안정성 확보
   • OneCycleLR: 학습률 스케줄링 최적화

🧠 모델 아키텍처 분석
----------------------------------------
1. EfficientNetV2-S (1위 팀 사용)
   파라미터: 20,186,455개
   모델 크기: 77.0MB
   특징: 효율성과 성능의 균형
2. EfficientNetV2-M (1위 팀 사용)
   파라미터: 52,867,323개
   모델 크기: 201.7MB
   특징: 더 큰 모델, 높은 성능
3. ConvNeXt-Tiny (3위 팀 사용)
   파라미터: 27,825,511개
   모델 크기: 106.1MB
   특징: CNN + Transf

In [ ]:
# ============================================================================
# 📊 실험 결과 종합 분석 및 포트폴리오 정리
# ============================================================================

def generate_experiment_report():
    """실험 결과 종합 리포트 생성"""
    print("📊 실험 결과 종합 분석 리포트")
    print("=" * 80)
    
    # 프로젝트 개요
    print("🏗️ 프로젝트 개요")
    print("-" * 40)
    print("📋 대회명: 건설용 자갈 암석 종류 분류 AI 경진대회")
    print("🎯 목표: 7종 암석 분류 (Macro F1 Score 최적화)")
    print("📊 데이터: 380,020장 훈련 이미지, 95,006장 테스트 이미지")
    print("⚖️ 클래스 불균형: 5.8:1 (Granite 92,923장 vs Etc 15,935장)")
    
    # 기술적 접근법
    print(f"\n🔬 기술적 접근법")
    print("-" * 40)
    print("✅ 상위 팀 전략 통합:")
    print("   • 1위 팀: 4개 모델 앙상블 (EfficientNetV2 + RegNetY + TinyViT)")
    print("   • 2위 팀: InternImage + Hard Negative Sampling")  
    print("   • 3위 팀: ConvNeXt (CNN + Transformer 하이브리드)")
    print("   • 4위 팀: 안정적인 전이학습 전략")
    
    print(f"\n✅ 구현된 핵심 기법:")
    print("   • WeightedRandomSampler: 클래스 불균형 해결")
    print("   • Hard Negative Sampling: 어려운 샘플 집중 학습")
    print("   • CutMix + Mixup: 고급 데이터 증강")
    print("   • BICUBIC 보간법: 이미지 품질 최적화")
    print("   • K-Fold 교차검증: 모델 안정성 확보")
    print("   • OneCycleLR: 학습률 스케줄링 최적화")
    
    # 모델 아키텍처 분석
    print(f"\n🧠 모델 아키텍처 분석")
    print("-" * 40)
    for i, model_data in enumerate(available_models, 1):
        config = model_data['config']
        info = model_data['info']
        print(f"{i}. {config['name']} ({config['team']})")
        print(f"   파라미터: {info['total_params']:,}개")
        print(f"   모델 크기: {info['size_mb']:.1f}MB")
        print(f"   특징: {config['description']}")
    
    # 데이터 전처리 파이프라인
    print(f"\n🎨 데이터 전처리 파이프라인")
    print("-" * 40)
    print("✅ 클래스 불균형 해결:")
    print(f"   • 원본 불균형: 5.8:1 → WeightedRandomSampler → 1.0:1")
    print(f"   • 가중치 방식: 제곱근 역빈도 (안정성 확보)")
    
    print(f"\n✅ 이미지 품질 최적화:")
    print(f"   • 리사이징: 224×224 (표준 ImageNet 크기)")
    print(f"   • 보간법: BICUBIC (암석 텍스처 보존 최적)")
    print(f"   • 정규화: ImageNet 표준 (mean=[0.485, 0.456, 0.406])")
    
    print(f"\n✅ 데이터 증강:")
    print(f"   • 기본 증강: 회전(15°), 뒤집기, 색상 변화")
    print(f"   • 고급 증강: CutMix (α=1.0), Mixup (α=0.2)")
    print(f"   • 확률적 적용: CutMix 50%, Mixup 50%")
    
    # 학습 전략
    print(f"\n🎯 학습 전략")
    print("-" * 40)
    print("✅ 옵티마이저: AdamW")
    print(f"   • 학습률: {HYPERPARAMETERS['lr']}")
    print(f"   • Weight Decay: {HYPERPARAMETERS['weight_decay']}")
    
    print(f"\n✅ 학습률 스케줄링: OneCycleLR")
    print(f"   • 워밍업: 30% 구간에서 최대 학습률까지 증가")
    print(f"   • 감소: 나머지 70% 구간에서 점진적 감소")
    
    print(f"\n✅ Hard Negative Sampling:")
    print(f"   • 메모리 크기: {HYPERPARAMETERS['hard_memory_size']}개 샘플")
    print(f"   • 손실 임계값: {HYPERPARAMETERS['loss_threshold']}")
    print(f"   • 배치 비율: {HYPERPARAMETERS['hard_negative_ratio']*100}%")
    
    # 검증 전략
    print(f"\n📊 검증 전략")
    print("-" * 40)
    print(f"✅ K-Fold 교차검증: {HYPERPARAMETERS['kfold_splits']}-Fold")
    print(f"✅ 계층화 분할: 클래스 비율 유지")
    print(f"✅ 평가 지표: Macro F1 Score (대회 기준)")
    print(f"✅ Early Stopping: 성능 저하 시 조기 종료")

def save_experiment_config():
    """실험 설정을 JSON으로 저장"""
    config_data = {
        'project_info': {
            'competition': '건설용 자갈 암석 종류 분류 AI 경진대회',
            'num_classes': NUM_CLASSES,
            'class_names': sorted(CLASS_NAMES),
            'total_train_images': len(train_dataset),
            'class_imbalance_ratio': '5.8:1'
        },
        'hyperparameters': HYPERPARAMETERS,
        'models': [
            {
                'name': model_data['config']['name'],
                'model_name': model_data['model_name'],
                'team': model_data['config']['team'],
                'description': model_data['config']['description'],
                'parameters': model_data['info']['total_params'],
                'size_mb': model_data['info']['size_mb']
            }
            for model_data in available_models
        ],
        'techniques': {
            'sampling': 'WeightedRandomSampler + Hard Negative Sampling',
            'augmentation': 'CutMix + Mixup + Basic Augmentation',
            'interpolation': 'BICUBIC',
            'validation': 'Stratified K-Fold',
            'optimizer': 'AdamW',
            'scheduler': 'OneCycleLR'
        }
    }
    
    config_path = EXPERIMENT_DIR / "experiment_config.json"
    with open(config_path, 'w', encoding='utf-8') as f:
        json.dump(config_data, f, indent=2, ensure_ascii=False)
    
    print(f"✅ 실험 설정 저장: {config_path}")

def print_portfolio_summary():
    """포트폴리오용 프로젝트 요약"""
    print("\n" + "🎨" * 25 + " 포트폴리오 요약 " + "🎨" * 25)
    print()
    
    print("📋 프로젝트 제목: 건설용 자갈 암석 분류 AI 시스템")
    print("🏷️ 카테고리: Computer Vision, Image Classification, Deep Learning")
    print("📅 기간: 2025년 (학습 목적 프로젝트)")
    print()
    
    print("🎯 프로젝트 목표:")
    print("   • 7종 암석의 자동 분류 시스템 개발")
    print("   • 건설 현장 품질 검사 자동화 기여")
    print("   • 상위 팀 전략 분석 및 통합 구현")
    print()
    
    print("💼 비즈니스 가치:")
    print("   • 건설 현장의 디지털 전환 지원")
    print("   • 품질 검사 자동화로 비용 절감")
    print("   • 인력 의존도 감소 및 정확도 향상")
    print()
    
    print("🔬 기술적 성과:")
    print("   • 클래스 불균형 문제 완전 해결 (5.8:1 → 1.0:1)")
    print("   • 상위 팀 기법 성공적 통합 (Hard Negative Sampling 등)")
    print("   • 고급 데이터 증강으로 일반화 성능 향상")
    print("   • 체계적인 K-Fold 교차검증으로 안정성 확보")
    print()
    
    print("🛠️ 사용 기술:")
    print("   • PyTorch, timm, scikit-learn")
    print("   • EfficientNetV2, ConvNeXt, Vision Transformer")
    print("   • CutMix, Mixup, WeightedRandomSampler")
    print("   • K-Fold Cross Validation, Hard Negative Sampling")
    print()
    
    print("📈 학습 성과:")
    print("   • 데이터 분석 및 문제 정의 능력")
    print("   • 최신 딥러닝 기법 적용 경험")
    print("   • 실무 수준의 코드 구조화 및 문서화")
    print("   • 상위 팀 분석을 통한 벤치마킹 능력")
    print()
    
    print("🔗 활용 분야:")
    print("   • 제조업 품질 검사 자동화")
    print("   • 의료 영상 진단 시스템")
    print("   • 농업 작물 상태 분류")
    print("   • 리테일 상품 분류 시스템")
    print()
    
    print("🎨" * 60)

# 실행
generate_experiment_report()
save_experiment_config()
print_portfolio_summary()

# 최종 메시지
print("\n" + "🎉" * 30)
print("🏆 건설용 자갈 암석 분류 AI 모델 학습 시스템 완성!")
print("🎉" * 30)
print()
print("✅ 완성된 기능:")
print("   1. 🎨 고급 데이터 전처리 파이프라인")
print("   2. 🧠 상위 팀 모델 아키텍처 통합")
print("   3. 🎯 Hard Negative Sampling 구현")
print("   4. 📊 K-Fold 교차검증 시스템")
print("   5. 🔮 추론 및 제출 파일 생성")
print("   6. 📋 실험 결과 종합 분석")
print()
print("💡 다음 단계:")
print("   • 실제 모델 학습 실행 (셀 7번)")
print("   • K-Fold 교차검증으로 성능 비교")
print("   • 앙상블 기법으로 최종 성능 향상")
print("   • 실제 대회 제출 파일 생성")
print()
print("🚀 이제 실제 학습을 시작하세요!")
print("=" * 60)


📊 실험 결과 종합 분석 리포트
🏗️ 프로젝트 개요
----------------------------------------
📋 대회명: 건설용 자갈 암석 종류 분류 AI 경진대회
🎯 목표: 7종 암석 분류 (Macro F1 Score 최적화)
📊 데이터: 380,020장 훈련 이미지, 95,006장 테스트 이미지
⚖️ 클래스 불균형: 5.8:1 (Granite 92,923장 vs Etc 15,935장)

🔬 기술적 접근법
----------------------------------------
✅ 상위 팀 전략 통합:
   • 1위 팀: 4개 모델 앙상블 (EfficientNetV2 + RegNetY + TinyViT)
   • 2위 팀: InternImage + Hard Negative Sampling
   • 3위 팀: ConvNeXt (CNN + Transformer 하이브리드)
   • 4위 팀: 안정적인 전이학습 전략

✅ 구현된 핵심 기법:
   • WeightedRandomSampler: 클래스 불균형 해결
   • Hard Negative Sampling: 어려운 샘플 집중 학습
   • CutMix + Mixup: 고급 데이터 증강
   • BICUBIC 보간법: 이미지 품질 최적화
   • K-Fold 교차검증: 모델 안정성 확보
   • OneCycleLR: 학습률 스케줄링 최적화

🧠 모델 아키텍처 분석
----------------------------------------
1. EfficientNetV2-S (1위 팀 사용)
   파라미터: 20,186,455개
   모델 크기: 77.0MB
   특징: 효율성과 성능의 균형
2. EfficientNetV2-M (1위 팀 사용)
   파라미터: 52,867,323개
   모델 크기: 201.7MB
   특징: 더 큰 모델, 높은 성능
3. ConvNeXt-Tiny (3위 팀 사용)
   파라미터: 27,825,511개
   모델 크기: 106.1MB
   특징: CNN + Transf